In [1]:
from ortools.linear_solver import pywraplp

In [40]:
#data
days = 5
slots_per_day = 4
nb_resources = 3
nb_projects = 4

squash_factor= 1.2

requested_slots_per_project = [4,5,3,4]

total_time_slots = days * slots_per_day
print(total_time_slots)

20


In [41]:
# Create the MIP solver
solver = pywraplp.Solver.CreateSolver('SCIP') # MIXED_INTEGER_PROGRAMMING

In [42]:
# variables definition

assignation = {}
for r in range(nb_resources):
    for p in range(nb_projects):
        for t in range(total_time_slots):
            assignation[(r, p, t)] = solver.BoolVar(f"assignation_r{r}_p{p}_t{t}")
zs = {}
zf = {}
for p in range(nb_projects):
    zs[(p)] = solver.IntVar(0, total_time_slots, f"zs_p{p}")
    zf[(p)] = solver.IntVar(0, total_time_slots, f"zf_p{p}")

y = solver.IntVar(0, total_time_slots, f"y")

In [43]:
print("Number of variables =", solver.NumVariables())

Number of variables = 249


In [44]:
# Define the objective function
objective = solver.Objective()
objective.SetCoefficient(y, 1)
objective.SetMinimization()


In [45]:
# constraints
# Define constraint: Total working time for each project is equal to 10
for p in range(nb_projects):
    solver.Add(sum(assignation[(r, p, t)] for r in range(nb_resources) for t in range(total_time_slots)) == requested_slots_per_project[p])


In [46]:
for p in range(nb_projects):
    for t in range(total_time_slots):
        solver.Add(t * assignation[(r,p,t)] <= zf[p])

for p in range(nb_projects):
    for t in range(total_time_slots):
        solver.Add(zs[p] <= t*assignation[(r,p,t)]+(1-assignation[(r,p,t)])*99*total_time_slots)

# constraint assign lower bound of y
for r in range(nb_resources):
    for p in range(nb_projects):
        for t in range(total_time_slots):
            solver.Add(t * assignation[(r,p,t)] <= y)

for p in range(nb_projects):
    solver.Add(zf[p]-zs[p] <= requested_slots_per_project[p]*squash_factor)

In [47]:
print(solver.NumConstraints())

408


In [48]:
solver.Solve()

0

In [53]:
print('Objective value =', solver.Objective().Value())
print('x =', zf[0].solution_value())
print('x =', zf[1].solution_value())
print('x =', zf[2].solution_value())
print('x =', zf[3].solution_value())
print('x =', y.solution_value())



Objective value = 1.0
x = 4.0
x = 6.0
x = 3.0
x = 4.0
x = 1.0
